In [1]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [3]:
dataset=pd.get_dummies(dataset,drop_first=True)
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [4]:
dataset=dataset.drop("User ID",axis=1)

In [5]:
dataset["Purchased"].value_counts()

0    257
1    143
Name: Purchased, dtype: int64

In [6]:
indep=dataset[["Age","EstimatedSalary","Gender_Male"]]
indep.shape

(400, 3)

In [7]:
dep=dataset["Purchased"]
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [8]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [10]:
from sklearn.tree import DecisionTreeClassifier
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

from sklearn.model_selection import GridSearchCV

param_grid = {'criterion':['gini','entropy'],
              'max_features': ['auto','sqrt','log2'],
              'splitter':['best','random']} 



grid = GridSearchCV(DecisionTreeClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
   
# fitting the model for grid search 
grid.fit(X_train, y_train) 



C:\Users\SONY\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    5.8s finished
C:\Users\SONY\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=None,
                                              splitter='best'),
             iid='warn', n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'sqrt', 'log2'],
       

In [11]:
# print best parameter after tuning 
#print(grid.best_params_) 
re=grid.cv_results_
#print(re)
grid_predictions = grid.predict(X_test) 
   

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)



# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)




In [12]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter {'criterion': 'entropy', 'max_features': 'log2', 'splitter': 'best'}: 0.9031880225910075


In [13]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[78  7]
 [ 6 43]]


In [14]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.93      0.92      0.92        85
           1       0.86      0.88      0.87        49

    accuracy                           0.90       134
   macro avg       0.89      0.90      0.90       134
weighted avg       0.90      0.90      0.90       134



In [15]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.8975990396158462

In [16]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002998,6.836514e-07,0.003998,7.370010e-07,gini,auto,best,"{'criterion': 'gini', 'max_features': 'auto', ...",0.781933,0.822067,0.802404,0.801983,0.016417,11
1,0.002000,1.251540e-06,0.003997,2.464940e-06,gini,auto,random,"{'criterion': 'gini', 'max_features': 'auto', ...",0.796667,0.787980,0.851704,0.812001,0.028141,8
2,0.003996,2.825019e-03,0.012992,1.343268e-02,gini,sqrt,best,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.840224,0.843509,0.839629,0.841114,0.001701,3
3,0.002000,2.247832e-07,0.003999,1.251540e-06,gini,sqrt,random,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.829715,0.799424,0.828889,0.819421,0.014064,6
4,0.002998,8.152696e-04,0.007662,4.705277e-04,gini,log2,best,"{'criterion': 'gini', 'max_features': 'log2', ...",0.819259,0.808666,0.851704,0.826488,0.018252,5
5,0.022319,2.873881e-02,0.005665,2.356515e-03,gini,log2,random,"{'criterion': 'gini', 'max_features': 'log2', ...",0.774074,0.799424,0.768742,0.780697,0.013347,12
6,0.002665,4.706399e-04,0.004997,1.413493e-03,entropy,auto,best,"{'criterion': 'entropy', 'max_features': 'auto...",0.807010,0.821710,0.873425,0.833845,0.028471,4
7,0.005665,3.770457e-03,0.006327,4.027311e-03,entropy,auto,random,"{'criterion': 'entropy', 'max_features': 'auto...",0.808847,0.810298,0.793809,0.804352,0.007437,10
8,0.004997,1.631707e-03,0.008329,4.779795e-03,entropy,sqrt,best,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.853824,0.809596,0.873425,0.845677,0.026606,2
9,0.006662,5.905191e-03,0.004663,9.417854e-04,entropy,sqrt,random,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.811743,0.806075,0.807497,0.808463,0.002416,9
